In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
import os
import errno
%matplotlib inline

# Import the data and clean it

## Store the data and import new data
This project is designed to be updated with new data frequently. Therefore, a file is used to store the historical data (old+new) for the model with predictions. Therefore it is necessary to create an empty file to store the historical data. And before creating, it is necessary to verify if the file already exists.

In [2]:
# Create initial empty file to store data
flags = os.O_CREAT | os.O_EXCL | os.O_WRONLY

try:
    file_handle = os.open('data1.csv', flags)
except OSError as e:
    if e.errno == errno.EEXIST:  # Failed as the file already exists.
        pass
    else:  # Something unexpected went wrong so reraise the exception.
        raise
else:  # No exception, so the file must have been created successfully.
    with os.fdopen(file_handle, 'w') as file_obj:
        # Using `os.fdopen` converts the handle to an object that acts like a
        # regular Python file object, and the `with` context manager means the
        # file will be automatically closed when we're done with it.
        file_obj.write("Initializing the table") 

In [7]:
data.shape

(11964, 58)

In [3]:
# Import the new data, preferably only the new data
# However, also old data can be uploaded again
data=pd.read_excel('HIS-VAP (dat).xlsx',header=None)
data.head(12)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,NaN,Ini.Datos,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,...,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00
1,NaN,Fin.Datos,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,...,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Servidor,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,...,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi
4,NaN,Tag,VAP:Valdepero.Val01.Ambient.Temperature,VAP:Valdepero.Val01.Ambient.WindDir,VAP:Valdepero.Val01.Ambient.WindSpeed,VAP:Valdepero.Val01.Controller.HubTemperature,VAP:Valdepero.Val01.Controller.TopTemperature,VAP:Valdepero.Val01.Electrical.ActivePower,VAP:Valdepero.Val01.Electrical.CosPhi,VAP:Valdepero.Val01.Electrical.CurrentPhase1,...,VAP:Valdepero.Val01.Meteorological.WindSpeed,VAP:Valdepero.Val01.Nacelle.Direction,VAP:Valdepero.Val01.Nacelle.Temperature,VAP:Valdepero.Val01.Rotor.BladesPitchAngle1,VAP:Valdepero.Val01.Rotor.HydraulicOilPressure,VAP:Valdepero.Val01.Rotor.HydraulicOilTempera...,VAP:Valdepero.Val01.Rotor.RPM,VAP:Valdepero.Val01.RotorSystem.BladesPitchAngle,VAP:Valdepero.Val01.RotorSystem.RotorRPM,VAP:Valdepero.Val01.Spinner.Temperature
5,NaN,NaN,Actual ambient temperature,Absolute wind direction,Actual windspeed,Actual Hub controller temp.,Actual top controller temp.,Electrical Active Power Aero 01 VESTAS V90,Electrical CosPhi Aero 01 VESTAS V90,Electrical Current Phase1 Aero 01 VESTAS V90,...,Meteorological Wind Speed Aero 01 VESTAS V90,Actual direction of nacelle,Actual nacelle Temp.,Rotor Blades Pitch Angle Aero 01 VESTAS V90,Rotor Hydraulic Oil Pressure Aero 01 VESTAS V90,Rotor Hydraulic Oil Temperature Aero 01 VESTAS...,Rotor RPM rpm Aero 01 VESTAS V90,Actual pitch angle,Actual rotor rpm,Actual spinner temperature
6,NaN,NaN,VAP:Valdepero.Val01.Ambient.Temperature [°C],VAP:Valdepero.Val01.Ambient.WindDir [°],VAP:Valdepero.Val01.Ambient.WindSpeed [m/s],VAP:Valdepero.Val01.Controller.HubTemperature ...,VAP:Valdepero.Val01.Controller.TopTemperature ...,VAP:Valdepero.Val01.Electrical.ActivePower [kW],VAP:Valdepero.Val01.Electrical.CosPhi [ ],VAP:Valdepero.Val01.Electrical.CurrentPhase1 [A],...,VAP:Valdepero.Val01.Meteorological.WindSpeed ...,VAP:Valdepero.Val01.Nacelle.Direction [º],VAP:Valdepero.Val01.Nacelle.Temperature [ºC],VAP:Valdepero.Val01.Rotor.BladesPitchAngle1 [º],VAP:Valdepero.Val01.Rotor.HydraulicOilPressure...,VAP:Valdepero.Val01.Rotor.HydraulicOilTemperat...,VAP:Valdepero.Val01.Rotor.RPM [rpm],VAP:Valdepero.Val01.RotorSystem.BladesPitchAng...,VAP:Valdepero.Val01.RotorSystem.RotorRPM [rpm],VAP:Valdepero.Val01.Spinner.Temperature [°C]
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Unidades,°C,°,m/s,°C,°C,kW,,A,...,m/s,º,ºC,º,bar,ºC,rpm,°,rpm,°C
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create column headers
col_header=data.loc[4].str[15:] #header starts in row four and we need from the character 15 onwards.
col_header[0]='measure_time'# assign value to the column with the date
col_header[1]='minute_counter'#assign value to the column with counter
col_header

0                             measure_time
1                           minute_counter
2                Val01.Ambient.Temperature
3                    Val01.Ambient.WindDir
4                  Val01.Ambient.WindSpeed
5          Val01.Controller.HubTemperature
6          Val01.Controller.TopTemperature
7             Val01.Electrical.ActivePower
8                  Val01.Electrical.CosPhi
9           Val01.Electrical.CurrentPhase1
10          Val01.Electrical.CurrentPhase2
11          Val01.Electrical.CurrentPhase3
12              Val01.Electrical.Frequency
13          Val01.Electrical.ReactivePower
14          Val01.Electrical.VoltagePhase1
15          Val01.Electrical.VoltagePhase2
16          Val01.Electrical.VoltagePhase3
17           Val01.Gear.BearingTemperature
18               Val01.Gear.OilTemperature
19      Val01.Generator.BearingTemperature
20       Val01.Generator.Phase1Temperature
21       Val01.Generator.Phase2Temperature
22       Val01.Generator.Phase3Temperature
23         

In [5]:
# duplicate the data original data to prevent corrupting it
data1=data

# assign the column headers to the data
data1=data1.rename(columns=col_header)
data1.head(12)

,measure_time,minute_counter,Val01.Ambient.Temperature,Val01.Ambient.WindDir,Val01.Ambient.WindSpeed,Val01.Controller.HubTemperature,Val01.Controller.TopTemperature,Val01.Electrical.ActivePower,Val01.Electrical.CosPhi,Val01.Electrical.CurrentPhase1,...,Val01.Meteorological.WindSpeed,Val01.Nacelle.Direction,Val01.Nacelle.Temperature,Val01.Rotor.BladesPitchAngle1,Val01.Rotor.HydraulicOilPressure,Val01.Rotor.HydraulicOilTemperature,Val01.Rotor.RPM,Val01.RotorSystem.BladesPitchAngle,Val01.RotorSystem.RotorRPM,Val01.Spinner.Temperature
0,NaN,Ini.Datos,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,...,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00,01/01/19 00:00:00
1,NaN,Fin.Datos,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,...,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00,25/03/19 00:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Servidor,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,...,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi,uwrenpi
4,NaN,Tag,VAP:Valdepero.Val01.Ambient.Temperature,VAP:Valdepero.Val01.Ambient.WindDir,VAP:Valdepero.Val01.Ambient.WindSpeed,VAP:Valdepero.Val01.Controller.HubTemperature,VAP:Valdepero.Val01.Controller.TopTemperature,VAP:Valdepero.Val01.Electrical.ActivePower,VAP:Valdepero.Val01.Electrical.CosPhi,VAP:Valdepero.Val01.Electrical.CurrentPhase1,...,VAP:Valdepero.Val01.Meteorological.WindSpeed,VAP:Valdepero.Val01.Nacelle.Direction,VAP:Valdepero.Val01.Nacelle.Temperature,VAP:Valdepero.Val01.Rotor.BladesPitchAngle1,VAP:Valdepero.Val01.Rotor.HydraulicOilPressure,VAP:Valdepero.Val01.Rotor.HydraulicOilTempera...,VAP:Valdepero.Val01.Rotor.RPM,VAP:Valdepero.Val01.RotorSystem.BladesPitchAngle,VAP:Valdepero.Val01.RotorSystem.RotorRPM,VAP:Valdepero.Val01.Spinner.Temperature
5,NaN,NaN,Actual ambient temperature,Absolute wind direction,Actual windspeed,Actual Hub controller temp.,Actual top controller temp.,Electrical Active Power Aero 01 VESTAS V90,Electrical CosPhi Aero 01 VESTAS V90,Electrical Current Phase1 Aero 01 VESTAS V90,...,Meteorological Wind Speed Aero 01 VESTAS V90,Actual direction of nacelle,Actual nacelle Temp.,Rotor Blades Pitch Angle Aero 01 VESTAS V90,Rotor Hydraulic Oil Pressure Aero 01 VESTAS V90,Rotor Hydraulic Oil Temperature Aero 01 VESTAS...,Rotor RPM rpm Aero 01 VESTAS V90,Actual pitch angle,Actual rotor rpm,Actual spinner temperature
6,NaN,NaN,VAP:Valdepero.Val01.Ambient.Temperature [°C],VAP:Valdepero.Val01.Ambient.WindDir [°],VAP:Valdepero.Val01.Ambient.WindSpeed [m/s],VAP:Valdepero.Val01.Controller.HubTemperature ...,VAP:Valdepero.Val01.Controller.TopTemperature ...,VAP:Valdepero.Val01.Electrical.ActivePower [kW],VAP:Valdepero.Val01.Electrical.CosPhi [ ],VAP:Valdepero.Val01.Electrical.CurrentPhase1 [A],...,VAP:Valdepero.Val01.Meteorological.WindSpeed ...,VAP:Valdepero.Val01.Nacelle.Direction [º],VAP:Valdepero.Val01.Nacelle.Temperature [ºC],VAP:Valdepero.Val01.Rotor.BladesPitchAngle1 [º],VAP:Valdepero.Val01.Rotor.HydraulicOilPressure...,VAP:Valdepero.Val01.Rotor.HydraulicOilTemperat...,VAP:Valdepero.Val01.Rotor.RPM [rpm],VAP:Valdepero.Val01.RotorSystem.BladesPitchAng...,VAP:Valdepero.Val01.RotorSystem.RotorRPM [rpm],VAP:Valdepero.Val01.Spinner.Temperature [°C]
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Unidades,°C,°,m/s,°C,°C,kW,,A,...,m/s,º,ºC,º,bar,ºC,rpm,°,rpm,°C
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# clean data & columns
data1=data1.drop([0,1,2,3,4,5,6,7,8,9,10]) #delete rows with no added value.
data1.columns = data1.columns.str.strip().str.lower().str.replace('.', '_') #more convenient "_" than "."
data1['measure_time']=data1['measure_time'].astype('str') #convert to string to slice it to only Hours and minutes
data1.measure_time=data1.measure_time.str[:16]
data1.head(5)

,measure_time,minute_counter,val01_ambient_temperature,val01_ambient_winddir,val01_ambient_windspeed,val01_controller_hubtemperature,val01_controller_toptemperature,val01_electrical_activepower,val01_electrical_cosphi,val01_electrical_currentphase1,...,val01_meteorological_windspeed,val01_nacelle_direction,val01_nacelle_temperature,val01_rotor_bladespitchangle1,val01_rotor_hydraulicoilpressure,val01_rotor_hydraulicoiltemperature,val01_rotor_rpm,val01_rotorsystem_bladespitchangle,val01_rotorsystem_rotorrpm,val01_spinner_temperature
11,2019-01-01 00:00,0,10.75,54.7,9.2,21,35,1605.4,1,1342.6,...,10.3533,51.038,23,-1.2378,194.025,24.9195,14.7553,-1.758,14.81,12
12,2019-01-01 00:10,600,10,54.81,9.5,21,35,1679.86,1,1429.16,...,10.4903,50.9975,23,-0.742385,190.137,24.8986,14.7452,-1.699,14.88,12
13,2019-01-01 00:20,1200,10,53.14,8.5,21,35,1527.05,1,1298,...,9.8,54.2,23,-1.1945,190.85,24.8776,14.735,-1.572,14.8082,12
14,2019-01-01 00:30,1800,10,54.4,8.87,21,35,1378.04,1,1209.4,...,9.15333,54.2,23.8044,-1.40358,184,24.8566,14.7248,-1.95133,14.8888,12
15,2019-01-01 00:40,2400,10,50.92,9.25,21,35,1315.3,1,1152,...,8.7,54.2,24,-1.61265,188.938,24.8357,14.7147,-2.133,14.8363,12


In [7]:
# Import the table with the previous models output data 
data2=pd.read_csv("~/Documents/Kschool/00_TFM/data1.csv")
data2.head(5) # result vary if it is the first time this model is initialized (empty) or has previously 

,Initializing the table


In [8]:
# Update the old data with the new data for the model.
data1.update(data2)

# Export the data
export_csv=data1.to_csv('/Users/juanfatri/Documents/Kschool/00_TFM/data1.csv',index=None, header=True)

In [9]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11953 entries, 11 to 11963
Data columns (total 58 columns):
measure_time                            11953 non-null object
minute_counter                          11953 non-null object
val01_ambient_temperature               11953 non-null object
val01_ambient_winddir                   11953 non-null object
val01_ambient_windspeed                 11953 non-null object
val01_controller_hubtemperature         11953 non-null object
val01_controller_toptemperature         11953 non-null object
val01_electrical_activepower            11953 non-null object
val01_electrical_cosphi                 11953 non-null object
val01_electrical_currentphase1          11953 non-null object
val01_electrical_currentphase2          11953 non-null object
val01_electrical_currentphase3          11953 non-null object
val01_electrical_frequency              11953 non-null object
val01_electrical_reactivepower          11953 non-null object
val01_electrical_volta

In [10]:
data1.head(5)

,measure_time,minute_counter,val01_ambient_temperature,val01_ambient_winddir,val01_ambient_windspeed,val01_controller_hubtemperature,val01_controller_toptemperature,val01_electrical_activepower,val01_electrical_cosphi,val01_electrical_currentphase1,...,val01_meteorological_windspeed,val01_nacelle_direction,val01_nacelle_temperature,val01_rotor_bladespitchangle1,val01_rotor_hydraulicoilpressure,val01_rotor_hydraulicoiltemperature,val01_rotor_rpm,val01_rotorsystem_bladespitchangle,val01_rotorsystem_rotorrpm,val01_spinner_temperature
11,2019-01-01 00:00,0,10.75,54.7,9.2,21,35,1605.4,1,1342.6,...,10.3533,51.038,23,-1.2378,194.025,24.9195,14.7553,-1.758,14.81,12
12,2019-01-01 00:10,600,10,54.81,9.5,21,35,1679.86,1,1429.16,...,10.4903,50.9975,23,-0.742385,190.137,24.8986,14.7452,-1.699,14.88,12
13,2019-01-01 00:20,1200,10,53.14,8.5,21,35,1527.05,1,1298,...,9.8,54.2,23,-1.1945,190.85,24.8776,14.735,-1.572,14.8082,12
14,2019-01-01 00:30,1800,10,54.4,8.87,21,35,1378.04,1,1209.4,...,9.15333,54.2,23.8044,-1.40358,184,24.8566,14.7248,-1.95133,14.8888,12
15,2019-01-01 00:40,2400,10,50.92,9.25,21,35,1315.3,1,1152,...,8.7,54.2,24,-1.61265,188.938,24.8357,14.7147,-2.133,14.8363,12
